[View in Colaboratory](https://colab.research.google.com/github/juliusfrost/Research-Paper-Implementations/blob/master/Neural_Arithmetic_Logic_Units.ipynb)

#Neural Arithmetic Logic Units

https://arxiv.org/abs/1808.00508

In [0]:
import tensorflow as tf

## The NALU layer

In [0]:
class NALU(tf.keras.layers.Layer):
  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(NALU, self).__init__(**kwargs)
    
  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    initializer = tf.keras.initializers.RandomUniform(minval=-1, maxval=1)
    self.G = self.add_weight(name='G', shape=shape, initializer=initializer, trainable=True)
    self.W_hat = self.add_weight(name='W_hat', shape=shape, initializer=initializer, trainable=True)
    self.M_hat = self.add_weight(name='M_hat', shape=shape, initializer=initializer, trainable=True)
    
    super(NALU, self).build(input_shape)
    
    
  def call(self, x):
    W = tf.tanh(self.W_hat) * tf.sigmoid(self.M_hat)
    a = tf.matmul(x, W)
    g = tf.sigmoid(tf.matmul(x, self.G))
    m = tf.exp(tf.matmul(tf.log(tf.abs(x)+0.0001), W))
    y = g*a + (1-g)*m
    return y
  
  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)
  
  def get_config(self):
    base_config = super(NALU, self).get_config()
    base_config['output_dim'] = self.output_dim
    
  @classmethod
  def from_config(cls, config):
    return cls(**config)


##Train

###seed

In [0]:
import numpy as np
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

###build model

In [0]:
x_train = np.random.random((10000, 2))*20-10
y_train = x_train[:,0] + x_train[:,1]


model = tf.keras.Sequential([
    NALU(8),
    NALU(1),
])
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='mse',       # mean squared error
              metrics=['mae'])

###train model

In [324]:
model.fit(x_train, y_train, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
10000/10000 [==============================] - 2s 175us/step - loss: 57.6385 - mean_absolute_error: 6.0693
Epoch 2/100
10000/10000 [==============================] - 2s 170us/step - loss: 45.6031 - mean_absolute_error: 5.2586
Epoch 3/100
10000/10000 [==============================] - 2s 173us/step - loss: 34.2341 - mean_absolute_error: 4.4540
Epoch 4/100
10000/10000 [==============================] - 2s 161us/step - loss: 22.7761 - mean_absolute_error: 3.5172
Epoch 5/100
10000/10000 [==============================] - 2s 152us/step - loss: 17.1324 - mean_absolute_error: 2.9011
Epoch 6/100
10000/10000 [==============================] - 2s 153us/step - loss: 11.5059 - mean_absolute_error: 2.4834
Epoch 7/100
10000/10000 [==============================] - 2s 165us/step - loss: 10.2021 - mean_absolute_error: 2.2094
Epoch 8/100
10000/10000 [==============================] - 2s 171us/step - loss: 7.3893 - mean_absolute_error: 1.9676
Epoch 9/100
10000/10000 [========================

##Test

Even though we trained on values in the range (-10,10), the model is great at extrapolating to values in the range (-100,100)

In [344]:
r = 100
x_test = np.random.random((10, 2)) *r*2 - r
y_test = x_test[:,0] + x_test[:,1]

for i,y in enumerate(model.predict(x_test)):
  print(str(round(x_test[i][0],2)) + ' + ' + str(round(x_test[i][1],2)) + '\npredicted: ' + str(round(y[0],2)) + '\nactual: ' + str(round(y_test[i],2)) + '\n')

-57.39 + -84.0
predicted: -141.01
actual: -141.39

18.67 + -56.1
predicted: -37.2
actual: -37.43

68.94 + 57.42
predicted: 126.01
actual: 126.35

3.09 + 28.73
predicted: 31.73
actual: 31.82

64.61 + 93.88
predicted: 158.03
actual: 158.5

-68.33 + 34.98
predicted: -33.48
actual: -33.34

87.28 + 86.11
predicted: 172.87
actual: 173.39

82.12 + 85.11
predicted: 166.74
actual: 167.23

-14.31 + 6.01
predicted: -8.32
actual: -8.3

96.28 + -48.89
predicted: 47.07
actual: 47.4

